# CW Attack Test

TJ Kim <br/>
2/2/21

#### Objective: 
Run CW Attack on generic federated learning setting.
See what the transfer rate is.

The goal is to build an infrastructure of C&W Attack Implementation.


In [1]:
cd '/home/ubuntu/FedAtk/' 

/home/ubuntu/FedAtk


### Load Relevant Libraries and Modules

Load the relevant libraries for the federated learning code.

In [2]:
import time
import yaml
        
from femnist_dataloader import Dataloader
from cnn_head import CNN_Head
from cnn_neck import CNN_Neck
from cnn_server import Server
from cnn_client import Client
from data_manager import DataManager

from utilities import freeze_layers
import numpy as np
import torch
import matplotlib.pyplot as plt
import random
import csv
import os
import pickle
from torch.autograd import Variable

import multiprocessing as mp

import queue

# Extra not from py file
from collections import OrderedDict 
import itertools

# Import C&W Attack Modules
import cw_attack.cw as cw

# Torch and plotting
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

### Directly Make victim NN Class

In [3]:
class Victim_NN(nn.Module):
    """
    Summary: 
    
    Pytorch NN module that takes pre-trained weights from layered personalized model
    We also load the data-loader and give test,attack functionality
    
    """
    
    def __init__(self, head_network, neck_network, dataloader):
        
        # Init attributes
        super(Victim_NN, self).__init__()
        self.head = head_network
        self.neck = neck_network
        self.dataloader = dataloader
        self.criterion = nn.NLLLoss()
        
        # Image min/max range per channel
        mean = [0.5]
        std = [0.5]

        self.inputs_box = (min((0 - m) / s for m, s in zip(mean, std)),
                           max((1 - m) / s for m, s in zip(mean, std)))
        
        # test_acc attributes
        self.orig_test_acc = None
        self.adv_test_acc = None
        
        self.orig_output_sim = None
        self.adv_output_sim = None
        
        # Attack attributes
        self.x_orig = None
        self.x_adv = None
        self.y_orig = None
        self.target = None
        
        self.softmax_orig = None
        self.output_orig = None
        self.softmax_adv = None
        self.output_adv = None
        
        self.orig_loss = None
        self.adv_loss = None
        self.orig_acc = None
        self.adv_acc = None
        
    def forward(self,x):
        x = self.neck.forward(x)
        x = self.head.forward(x)
        
        return x
    
    def forward_transfer(self, x_orig, x_adv, y_orig, y_adv,
                         true_labels, target, print_info = False):
        """
        Assume that input images are in pytorch tensor format
        """
        
        batch_size = y_orig.shape[0]
        
        # Forward Two Input Types
        h_adv = self.forward(x_adv)
        h_orig = self.forward(x_orig)
        h_adv_category = torch.argmax(h_adv,dim = 1)
        h_orig_category = torch.argmax(h_orig,dim = 1)
        
        # Record Different Parameters
        self.orig_test_acc = (h_orig_category == true_labels).float().sum()/batch_size
        self.adv_test_acc = (h_adv_category == true_labels).float().sum()/batch_size
        
        self.orig_output_sim = (h_orig_category == y_orig).float().sum()/batch_size
        self.adv_output_sim = (h_adv_category == y_adv).float().sum()/batch_size
        
        self.orig_target_achieve = (h_orig_category == target).float().sum()/batch_size
        self.adv_target_achieve = (h_adv_category == target).float().sum()/batch_size

        
        # Print Relevant Information
        if print_info:
            print("---- Attack Transfer:", "----\n")
            print("         Orig Test Acc:", self.orig_test_acc.item())
            print("          Adv Test Acc:", self.adv_test_acc.item())
            print("Orig Output Similarity:", self.orig_output_sim.item())
            print(" Adv Output Similarity:", self.adv_output_sim.item())
            print("       Orig Target Hit:", self.orig_target_achieve.item())
            print("        Adv Target Hit:", self.adv_target_achieve.item())
            
    def specs_from_xadv(self, print_info=False):
        
        self.softmax_orig = self.forward(self.x_orig)
        self.output_orig = torch.argmax(self.softmax_orig,dim=1)
        self.softmax_adv = self.forward(self.x_adv)
        self.output_adv = torch.argmax(self.softmax_adv,dim=1)
        
        # Record accuracy and loss
        self.orig_loss = self.criterion(self.softmax_orig, self.y_orig).item()
        self.adv_loss = self.criterion(self.softmax_adv, self.y_orig).item()
        self.orig_acc = (self.output_orig == self.y_orig).float().sum()/batch_size
        self.adv_acc = (self.output_adv == self.y_orig).float().sum()/batch_size
        
        # Add Perturbation Distance (L2 norm) - across each input
        # CURRENTLY BROKEN -- need to replace the code below.
        # self.norm = torch.norm(torch.sub(self.x_orig, self.x_adv, alpha=1),dim=(2,3))

        # Print Relevant Information
        if print_info:
            print("---- FGSM Batch Size:", batch_size, "----\n")
            print("Orig Target:", self.y_orig.tolist())
            print("Orig Output:", self.output_orig.tolist())
            print("ADV Output :", self.output_adv.tolist(),'\n')
            print("Orig Loss  :", self.orig_loss)
            print("ADV Loss   :", self.adv_loss,'\n')
            print("Orig Acc   :", self.orig_acc.item())
            print("ADV Acc    :", self.adv_acc.item())
            
    def CW_attack(self, batch_size = 10, target= -1, confidence=0.0, optimizer_lr=5e-4, 
                  iteration=1, x_val_mean = (0.5), x_val_std= (0.5), print_info=False):
        
        self.eval()
        
        image_data = self.dataloader.load_batch(batch_size)
        self.x_orig  = torch.Tensor(image_data['input']).reshape(batch_size,1,28,28).cuda()
        self.y_orig = torch.Tensor(image_data['label']).type(torch.LongTensor).cuda()
        self.target = target
        
        if target > -1:
            targeted = False
            targets = self.y_orig
        else:
            targeted = True
            targets = torch.ones(image_data['input'].size(0), dtype = torch.long).cuda() * target
            
        
        adversary = cw.L2Adversary(targeted=targeted,
                           confidence=confidence,
                           search_steps=iteration,
                           box= self.inputs_box,
                           optimizer_lr=optimizer_lr)
        
        self.x_adv = adversary(self, self.x_orig, targets, to_numpy=False).cuda()
        
        #assert isinstance(self.x_adv, torch.FloatTensor)
        assert self.x_adv.size() == self.x_orig.size()
        
        self.specs_from_xadv(print_info)
        
        
    def i_fgsm(self, batch_size = 10, target= -1, eps=0.03, alpha=1, 
               iteration=1, x_val_min=-1, x_val_max=1, print_info=False):
        """
        batch_size - number of images to adversarially perturb
        targetted - target class output we desire to alter all inputs into
        eps - max amount to add perturbations per pixel per iteration
        alpha - gradient scaling (increase minimum perturbation amount below epsilon)
        iteration - how many times to perturb
        x_val_min/max - NN input valid range to keep perturbations within
        """
        self.eval()
        
        # Load data to perturb
    
        image_data = self.dataloader.load_batch(batch_size)
        self.x_orig  = torch.Tensor(image_data['input']).reshape(batch_size,1,28,28)
        self.y_orig = torch.Tensor(image_data['label']).type(torch.LongTensor).cuda()
        self.target = target
        
        self.x_adv = Variable(self.x_orig, requires_grad=True)
        
        for i in range(iteration):
            
            h_adv = self.forward(self.x_adv)
            
            # Loss function based on target
            if target > -1:
                target_tensor = torch.LongTensor(self.y_orig.size()).fill_(target)
                target_tensor = Variable(cuda(target_tensor, self.cuda), requires_grad=False)
                cost = self.criterion(h_adv, target_tensor)
            else:
                cost = -self.criterion(h_adv, self.y_orig)

            self.zero_grad()

            if self.x_adv.grad is not None:
                self.x_adv.grad.data.fill_(0)
            cost.backward()

            self.x_adv.grad.sign_()
            self.x_adv = self.x_adv - alpha*self.x_adv.grad
            self.x_adv = where(self.x_adv > self.x_orig+eps, self.x_orig+eps, self.x_adv)
            self.x_adv = where(self.x_adv < self.x_orig-eps, self.x_orig-eps, self.x_adv)
            self.x_adv = torch.clamp(self.x_adv, x_val_min, x_val_max)
            self.x_adv = Variable(self.x_adv.data, requires_grad=True)

        self.specs_from_xadv(print_info)
        

### Transfer Attack For Set Configuration

As a demo, we will perform a transfer attack from client 0 to the other 7 clients in the system. The implementation is identical to the FGSM testbed.

In [4]:
# Dummy bit of code, the transfer attacks don't happen properly without this for some reason

filename = 'exp3_neck2_0_head3'

# Generate Head and Neck NN objects
mode = 'cuda'
head_nn = CNN_Head(mode)
neck_nn = CNN_Neck(mode)

# Which network to load and directory
i = 0
exp_path = "Results/federated_system/"+filename+"/"
nn_path = exp_path + filename + "_"

# Load pre-trained weights
head_path = nn_path + str(i) +"_head_network"
neck_path = nn_path + str(i) +"_neck_network"

head = torch.load(head_path)
neck = torch.load(neck_path)
    
head_edit = OrderedDict()
neck_edit = OrderedDict()

# Edit the ordered_dict key names to be torch compatible
for key in head.keys():
    head_edit["network."+key] = head[key]

for key in neck.keys():
    neck_edit["network."+key] = neck[key]

head_nn.load_state_dict(head_edit)
neck_nn.load_state_dict(neck_edit)

/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


<All keys matched successfully>

In [5]:

# Load Config File and Slie Indices
with open(r'config.yaml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
        
file_indices = [i for i in range(config['num_sets'])]
#random.shuffle(file_indices)
client_slice = len(file_indices)//config['num_clients']

# File names of FL trained setting
filenames = ["exp3_neck2_0_head3"]

# Matrix to Record Performance
orig_acc_transfers = np.zeros((1,config['num_clients']))
orig_similarities = np.zeros((1,config['num_clients']))
orig_target_hit = np.zeros((1,config['num_clients']))
adv_acc_transfers = np.zeros((1,config['num_clients']))
adv_similarities = np.zeros((1,config['num_clients']))
adv_target_hit = np.zeros((1,config['num_clients']))

# Attack Params
batch_size = 100
iteration = 50
target = 5

# IFGSM
eps = 0.3
alpha = 0.3

# CWA
confidence = 0
optimizer_lr = 5e-3

In [6]:
# Basic loader for generating the victim

def load_victim(idx, loader):
    # Load the corresponding head/neck network in victim nn module 
    
    mode = 'cuda'
    #head_nn = CNN_Head(mode)
    #neck_nn = CNN_Neck(mode)
    
    # Which network to load and directory
    exp_path = "Results/federated_system/"+filename+"/"
    nn_path = exp_path + filename + "_"

    # Load pre-trained weights
    head_path = nn_path + str(idx) +"_head_network"
    neck_path = nn_path + str(idx) +"_neck_network"

    head = torch.load(head_path)
    neck = torch.load(neck_path)

    head_edit = OrderedDict()
    neck_edit = OrderedDict()

    # Edit the ordered_dict key names to be torch compatible
    for key in head.keys():
        head_edit["network."+key] = head[key]

    for key in neck.keys():
        neck_edit["network."+key] = neck[key]

    head_nn.load_state_dict(head_edit)
    neck_nn.load_state_dict(neck_edit)
    
    return Victim_NN(head_nn,neck_nn,loader)

Perform the transfer attack below.

In [7]:
for source in range(1):
    
    # Bring in the data loader for this client
    loader = Dataloader(file_indices,[source*(client_slice),min((source+1)*(client_slice),35)])  
    loader.load_training_dataset()
    loader.load_testing_dataset()

    victim_source = load_victim(source,loader)

    # Generate adversarial Perturbations
    victim_source.CW_attack(batch_size = batch_size, target= target, confidence=confidence, 
                            optimizer_lr=optimizer_lr, iteration=iteration, x_val_mean = (0.5), 
                            x_val_std= (0.5), print_info=False)

    # Record relevant tensors
    x_orig = victim_source.x_orig
    y_orig = victim_source.output_orig
    y_true = victim_source.y_orig
    x_adv = victim_source.x_adv
    y_adv = victim_source.output_adv

    print("======== Source", source, "========")

    for dest in range(config['num_clients']):

        print("    ==== Dest", dest, "====")

        victim_dest = load_victim(dest,loader)

        # Compute Stats and record
        victim_dest.forward_transfer(x_orig,x_adv,y_orig,y_adv,y_true, target, print_info=True)

        orig_acc_transfers[source,dest] = victim_dest.orig_test_acc
        orig_similarities[source,dest] = victim_dest.orig_output_sim
        orig_target_hit[source,dest] = victim_dest.orig_target_achieve

        adv_acc_transfers[source,dest] = victim_dest.adv_test_acc
        adv_similarities[source,dest] = victim_dest.adv_output_sim
        adv_target_hit[source,dest] = victim_dest.adv_target_achieve

Loading  all_data_12_niid_0_keep_0_train_9.json
Loading  all_data_20_niid_0_keep_0_train_9.json
Loading  all_data_11_niid_0_keep_0_train_9.json
Loading  all_data_18_niid_0_keep_0_train_9.json
Using scale consts: [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
Using scale consts: [0.0005, 0.01, 0.01, 0.01, 0.01, 0.0005, 0.01, 0.01, 0.01, 0.01, 0.

Using scale consts: [3.90625e-06, 0.0020546875, 0.01140625, 0.0031796875000000002, 0.03390625, 3.90625e-06, 0.0020546875, 0.031093750000000003, 0.0044453125, 0.05078125, 3.90625e-06, 0.01421875, 0.0054296875, 0.006835937499999999, 3.90625e-06, 0.01140625, 3.90625e-06, 3.90625e-06, 0.01703125, 0.039531250000000004, 0.01421875, 0.01984375, 0.0026171874999999997, 0.01984375, 0.031093750000000003, 0.0061328125, 0.01140625, 0.0028984375, 0.031093750000000003, 0.00099609375, 0.0027578125, 0.045156249999999995, 0.01984375, 0.01703125, 0.01984375, 0.01140625, 0.01984375, 0.01140625, 0.008242187500000001, 0.01140625, 0.053593749999999996, 0.01421875, 0.01140625, 0.01421875, 0.01140625, 0.0010703125, 0.01140625, 3.90625e-06, 0.0054296875, 3.90625e-06, 0.0010703125, 0.0020546875, 0.0031796875000000002, 0.04234375, 3.90625e-06, 0.01421875, 0.0055703125, 0.031093750000000003, 0.01140625, 3.90625e-06, 0.01984375, 0.0054296875, 0.0078203125, 0.0054296875, 0.01421875, 0.0054296875, 0.01140625, 0.00099

Using scale consts: [1.220703125e-07, 0.002122802734375, 0.0112744140625, 0.003151123046875, 0.032719726562499996, 1.220703125e-07, 0.0021096191406250004, 0.0324560546875, 0.004377197265624999, 0.0517919921875, 1.220703125e-07, 0.014965820312500001, 0.005497802734375, 0.006789794921874998, 1.220703125e-07, 0.012241210937500002, 1.220703125e-07, 1.220703125e-07, 0.015932617187499998, 0.0404541015625, 0.014965820312500001, 0.021206054687500003, 0.0026105957031249995, 0.021206054687500003, 0.0324560546875, 0.0061350097656249995, 0.012768554687500001, 0.002830322265625, 0.030258789062500006, 0.0009998779296875, 0.002689697265625, 0.0441455078125, 0.021206054687500003, 0.0160205078125, 0.021206054687500003, 0.0100439453125, 0.020327148437499998, 0.010659179687500001, 0.008310302734375, 0.0100439453125, 0.05460449218749999, 0.014965820312500001, 0.010131835937500002, 0.0155810546875, 0.0100439453125, 0.001002197265625, 0.010483398437500001, 1.220703125e-07, 0.005497802734375, 1.220703125e-07

Using scale consts: [3.814697265625e-09, 0.002124931335449219, 0.011275787353515625, 0.0031495437622070313, 0.03276229858398437, 3.814697265625e-09, 0.002111747741699219, 0.032498626708984374, 0.00437506866455078, 0.05176589965820312, 3.814697265625e-09, 0.014923248291015627, 0.005499931335449218, 0.006787940979003905, 3.814697265625e-09, 0.012275543212890627, 3.814697265625e-09, 3.814697265625e-09, 0.015898284912109373, 0.04046646118164063, 0.014923248291015627, 0.021248626708984378, 0.002611625671386718, 0.021248626708984378, 0.032498626708984374, 0.006132881164550781, 0.012811126708984375, 0.0028281936645507813, 0.03022720336914063, 0.0009999961853027345, 0.002687568664550781, 0.04416610717773438, 0.021248626708984378, 0.01602188110351563, 0.021248626708984378, 0.010001373291015626, 0.020284576416015623, 0.010622100830078127, 0.008310234069824219, 0.010001373291015626, 0.05456192016601562, 0.014978179931640625, 0.010133209228515627, 0.015582427978515626, 0.010001373291015626, 0.0010

Using scale consts: [2.384185791015625e-10, 0.0021249957084655763, 0.011274499893188475, 0.0031494793891906738, 0.03276135444641112, 2.384185791015625e-10, 0.0021118121147155767, 0.03249991416931153, 0.004375004291534423, 0.05176461219787597, 2.384185791015625e-10, 0.014921960830688478, 0.005499995708465576, 0.0067878766059875474, 2.384185791015625e-10, 0.012274255752563478, 2.384185791015625e-10, 2.384185791015625e-10, 0.015896997451782226, 0.040465173721313474, 0.014921960830688478, 0.021249914169311525, 0.0026115612983703606, 0.021249914169311525, 0.03249991416931153, 0.006132816791534424, 0.012812414169311524, 0.0028281292915344237, 0.030225915908813482, 0.0009999997615814209, 0.0026875042915344236, 0.04416550636291504, 0.021249914169311525, 0.016020593643188478, 0.021249914169311525, 0.010000085830688475, 0.020283288955688476, 0.010620813369750976, 0.008310186862945556, 0.010000085830688475, 0.054560632705688464, 0.014976892471313476, 0.010131921768188477, 0.015581140518188475, 0.

...

Using scale consts: [5.820766091346741e-14, 0.0021249999989522623, 0.011274414083454757, 0.003149475098703988, 0.032761279365513464, 5.820766091346741e-14, 0.0021118158540921294, 0.03249999997904524, 0.004375000001047737, 0.051764635017607356, 5.820766091346741e-14, 0.01492187502095476, 0.005499999998952261, 0.006787873388384467, 5.820766091346741e-14, 0.012274267172906549, 5.820766091346741e-14, 5.820766091346741e-14, 0.01589693527901545, 0.04046508791157976, 0.01492187502095476, 0.021249999979045246, 0.0026115619071060783, 0.021249999979045246, 0.03249999997904524, 0.006132812501047738, 0.012812499979045242, 0.0028281250010477378, 0.03022583076963202, 0.0009999999999417922, 0.0026875000010477376, 0.04416549293091521, 0.021249999979045246, 0.01602050783345476, 0.021249999979045246, 0.010000000020954757, 0.020283203145954755, 0.010620834848377853, 0.008310182572458871, 0.010000000020954757, 0.05456054689595475, 0.014976806661579758, 0.010131835958454759, 0.015581054708454757, 0.0100000

Using scale consts: [3.637978807091713e-15, 0.0021249999999345165, 0.011274414063809673, 0.0031494750977217336, 0.03276127934586838, 3.637978807091713e-15, 0.0021118158531098752, 0.032499999998690327, 0.004375000000065483, 0.05176463500320096, 3.637978807091713e-15, 0.014921875001309674, 0.005499999999934516, 0.0067878733874022126, 3.637978807091713e-15, 0.012274267163738842, 3.637978807091713e-15, 3.637978807091713e-15, 0.015896935259370366, 0.040465087891934676, 0.014921875001309674, 0.02124999999869033, 0.002611561906123824, 0.02124999999869033, 0.032499999998690327, 0.0061328125000654835, 0.012812499998690328, 0.0028281250000654836, 0.03022583076046431, 0.000999999999996362, 0.0026875000000654834, 0.04416549294794095, 0.02124999999869033, 0.016020507813809676, 0.02124999999869033, 0.010000000001309673, 0.02028320312630967, 0.010620834860164904, 0.008310182571476617, 0.010000000001309673, 0.054560546876309665, 0.014976806641934673, 0.010131835938809675, 0.015581054688809673, 0.01000

Using scale consts: [2.2737367544323206e-16, 0.0021249999999959075, 0.011274414062581853, 0.0031494750976603426, 0.032761279344640556, 2.2737367544323206e-16, 0.0021118158530484842, 0.03249999999991815, 0.004375000000004093, 0.05176463500262797, 2.2737367544323206e-16, 0.014921875000081856, 0.0054999999999959075, 0.0067878733873408224, 2.2737367544323206e-16, 0.012274267162511024, 2.2737367544323206e-16, 2.2737367544323206e-16, 0.015896935258306256, 0.04046508789136169, 0.014921875000081856, 0.021249999999918147, 0.0026115619061033604, 0.021249999999918147, 0.03249999999991815, 0.006132812500004093, 0.012812499999918146, 0.0028281250000040926, 0.0302258307594002, 0.0009999999999997728, 0.0026875000000040924, 0.04416549294687684, 0.021249999999918147, 0.016020507812581856, 0.021249999999918147, 0.010000000000081853, 0.020283203125081854, 0.010620834859100797, 0.008310182571415225, 0.010000000000081853, 0.05456054687508184, 0.014976806640706854, 0.010131835937581855, 0.015581054687581853

Using scale consts: [1.4210854715202004e-17, 0.0021249999999997443, 0.011274414062505116, 0.003149475097656506, 0.03276127934456382, 1.4210854715202004e-17, 0.0021118158530446475, 0.03249999999999489, 0.004375000000000255, 0.05176463500255123, 1.4210854715202004e-17, 0.014921875000005117, 0.005499999999999744, 0.006787873387336985, 1.4210854715202004e-17, 0.012274267162434285, 1.4210854715202004e-17, 1.4210854715202004e-17, 0.015896935258321605, 0.04046508789128495, 0.014921875000005117, 0.021249999999994884, 0.0026115619060995236, 0.021249999999994884, 0.03249999999999489, 0.006132812500000256, 0.012812499999994885, 0.0028281250000002558, 0.03022583075932346, 0.0009999999999999857, 0.0026875000000002557, 0.04416549294689219, 0.021249999999994884, 0.01602050781251535, 0.021249999999994884, 0.010000000000005116, 0.020283203125005117, 0.010620834859024058, 0.00831018257141139, 0.010000000000005116, 0.054560546875005105, 0.014976806640630115, 0.010131835937505117, 0.015581054687505116, 0.

======== Source 0 ========
    ==== Dest 0 ====
---- Attack Transfer: ----

         Orig Test Acc: 0.8799999952316284
          Adv Test Acc: 0.0
Orig Output Similarity: 1.0
 Adv Output Similarity: 1.0
       Orig Target Hit: 0.05999999865889549
        Adv Target Hit: 0.05999999865889549
    ==== Dest 1 ====
---- Attack Transfer: ----

         Orig Test Acc: 0.8399999737739563
          Adv Test Acc: 0.6899999976158142
Orig Output Similarity: 0.8399999737739563
 Adv Output Similarity: 0.22999998927116394
       Orig Target Hit: 0.03999999910593033
        Adv Target Hit: 0.04999999701976776
    ==== Dest 2 ====
---- Attack Transfer: ----

         Orig Test Acc: 0.8199999928474426
          Adv Test Acc: 0.7199999690055847
Orig Output Similarity: 0.8299999833106995
 Adv Output Similarity: 0.1899999976158142
       Orig Target Hit: 0.04999999701976776
        Adv Target Hit: 0.019999999552965164
    ==== Dest 3 ====
---- Attack Transfer: ----

         Orig Test Acc: 0.81000000238418

In [8]:
print("orig_acc_transfers\n",np.round(orig_acc_transfers,3)[0])
print("orig_similarities\n",np.round(orig_similarities,3))
print("orig_target_hit\n",np.round(orig_target_hit,3))
print("adv_acc_transfers\n",np.round(adv_acc_transfers,3))
print("adv_similarities\n",np.round(adv_similarities,3))
print("adv_target_hit\n",np.round(adv_target_hit,3))

orig_acc_transfers
 [0.88 0.84 0.82 0.81 0.65 0.79 0.78 0.85]
orig_similarities
 [[1.   0.84 0.83 0.85 0.72 0.84 0.81 0.85]]
orig_target_hit
 [[0.06 0.04 0.05 0.05 0.05 0.06 0.06 0.05]]
adv_acc_transfers
 [[0.   0.69 0.72 0.63 0.6  0.71 0.71 0.72]]
adv_similarities
 [[1.   0.23 0.19 0.25 0.22 0.22 0.21 0.25]]
adv_target_hit
 [[0.06 0.05 0.02 0.03 0.06 0.05 0.04 0.02]]


In [9]:
victim_source.orig_acc

tensor(0.8800, device='cuda:0')

In [10]:
victim_source.adv_acc

tensor(0., device='cuda:0')